In [14]:
import numpy as np
import tensorflow as tf
from scipy.fftpack import dct
import cv2
from image_processor_gpu import *
@tf.function
def perform_blockwise_dct_tf(img):
    shape = tf.shape(img)
    x, y, c = shape[0], shape[1], shape[2]
    img_res = tf.reshape(img, [x // 8, 8, y // 8, 8, c])
    print(img_res)
    img_dct1 = tf.signal.dct(tf.transpose(img_res, [0, 1, 2, 4, 3]), norm='ortho')
    img_dct2 = tf.signal.dct(tf.transpose(img_dct1, [0, 2, 4, 3, 1]), norm='ortho')
    out = tf.reshape(tf.transpose(img_dct2, [0, 4, 1, 2, 3]), shape)
    return out


def perform_blockwise_dct(img):
    imsize = img.shape
    dct_blocks = np.zeros(imsize, dtype=img.dtype)
    for i in np.r_[:imsize[0]:8]:
        for j in np.r_[:imsize[1]:8]:
            dct_blocks[i:(i+8), j:(j+8), 0] = dct(dct(img[i:(i+8), j:(j+8), 0].T, norm='ortho').T, norm='ortho')
            dct_blocks[i:(i+8), j:(j+8), 1] = dct(dct(img[i:(i+8), j:(j+8), 1].T, norm='ortho').T, norm='ortho')
            dct_blocks[i:(i+8), j:(j+8), 2] = dct(dct(img[i:(i+8), j:(j+8), 2].T, norm='ortho').T, norm='ortho')
    return dct_blocks

if __name__ == "__main__":
    img = cv2.imread("test.png")
    img_yuv = cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
    
#     img = cv2.resize(img, None, fx=0.6, fy=0.6)
#     dct_matrix = dct_matrix_creator()
#     quantization_matrix_lc = np.load(os.path.join("quantization_tables", "Adobe_Photoshop__Save_As_00_lc.npy")).reshape(
#         8, 8)
#     quantization_matrix_cc = np.load(os.path.join("quantization_tables", "Adobe_Photoshop__Save_As_00_lc.npy")).reshape(
#         8, 8)
#     processed_img = background_processor(img, dct_matrix, quantization_matrix_lc, quantization_matrix_lc)
#     print(processed_img.max())
#     print(processed_img.min())
    out1 = perform_blockwise_dct(img_yuv.astype(np.float32))
    out2 = perform_blockwise_dct_tf(img_yuv.astype(np.float32))
    # There is a bit of error
    print(np.allclose(out1, out2, rtol=1e-5, atol=1e-6))

Tensor("Reshape:0", shape=(225, 8, 450, 8, 3), dtype=float32)
False
